# sveltejs/sveltekit

> 

In [1]:
#| default_exp sveltejs-sveltekit

In [2]:
#| hide
import nbdev; nbdev.nbdev_export()

In [3]:
#| export
from lovely_docs.settings import settings, GitSource
from lovely_docs.git import clone_repo
from lovely_docs.docs import (
    build_markdown_doc_tree,
    process_tree_depth_first,
    calculate_total_usage,
    calculate_cost,
    save_processed_documents
)
import llm
from llm.models import Usage
import anthropic
from pydantic import BaseModel, Field
from jinja2 import Environment, FileSystemLoader
import asyncio
import json
import shutil
from pathlib import Path
import langsmith as ls
import logging

In [4]:
#| export

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

#### Clone the repo

In [5]:
#| export

source = settings.sources["sveltejs/sveltekit"]
assert isinstance(source, GitSource)

commit, clone_dir = clone_repo(source)
source.commit = commit # Replace the literal commit (master) with the hash

#### Read the markdown files

In [6]:
tree = build_markdown_doc_tree(clone_dir/source.doc_dir)
# tree.model_dump()

#### Process the tree

In [7]:
# tree.subdirs = tree.subdirs[:2] # For testing process just the first 2 secstion (intro and runes)

In [8]:
processed_tree = await process_tree_depth_first(settings, tree, source.name)

+0.399s DEBUG lovely_docs/docs.py:111 llm_process_page Processing 10-getting-started/10-introduction.md
+0.312s DEBUG lovely_docs/docs.py:111 llm_process_page Processing 10-getting-started/20-creating-a-project.md
+0.015s DEBUG lovely_docs/docs.py:111 llm_process_page Processing 10-getting-started/25-project-types.md
+0.014s DEBUG lovely_docs/docs.py:111 llm_process_page Processing 10-getting-started/30-project-structure.md
+0.016s DEBUG lovely_docs/docs.py:111 llm_process_page Processing 10-getting-started/40-web-standards.md
+0.015s DEBUG lovely_docs/docs.py:111 llm_process_page Processing 10-getting-started/index.md
+0.016s DEBUG lovely_docs/docs.py:111 llm_process_page Processing 20-core-concepts/10-routing.md
+0.018s DEBUG lovely_docs/docs.py:111 llm_process_page Processing 20-core-concepts/20-load.md
+0.015s DEBUG lovely_docs/docs.py:111 llm_process_page Processing 20-core-concepts/30-form-actions.md
+0.015s DEBUG lovely_docs/docs.py:111 llm_process_page Processing 20-core-concep

#### Calculate cost

In [9]:
usage =calculate_total_usage(processed_tree)
print(f"\nTotal Usage:")
print(f"  Input tokens: {usage.input:,}")
print(f"  Output tokens: {usage.output:,}")
print(f"  Total tokens: {(usage.input + usage.output):,}")

cost, input_cost, output_cost = calculate_cost(usage, 1, 5)

print(f"Cost:")
print(f"  Total: ${cost:.2f}")
print(f"  Input: ${input_cost:.2f}")
print(f"  Output: ${output_cost:.2f}")


Total Usage:
  Input tokens: 269,275
  Output tokens: 68,611
  Total tokens: 337,886
Cost:
  Total: $0.61
  Input: $0.27
  Output: $0.34


#### save the results

In [10]:
save_processed_documents(source, settings.output_dir / source.name, processed_tree)

+82.701s DEBUG lovely_docs/docs.py:367 file_map Added . ->  (.)
+0.001s DEBUG lovely_docs/docs.py:367 file_map Added getting-started -> getting-started (10-getting-started)
+0.000s DEBUG lovely_docs/docs.py:375 file_map Added page getting-started/introduction -> introduction (10-getting-started/10-introduction.md)
+0.000s DEBUG lovely_docs/docs.py:375 file_map Added page getting-started/creating-a-project -> creating-a-project (10-getting-started/20-creating-a-project.md)
+0.000s DEBUG lovely_docs/docs.py:375 file_map Added page getting-started/project-types -> project-types (10-getting-started/25-project-types.md)
+0.000s DEBUG lovely_docs/docs.py:375 file_map Added page getting-started/project-structure -> project-structure (10-getting-started/30-project-structure.md)
+0.000s DEBUG lovely_docs/docs.py:375 file_map Added page getting-started/web-standards -> web-standards (10-getting-started/40-web-standards.md)
+0.000s DEBUG lovely_docs/docs.py:375 file_map Added page getting-started

In [11]:
processed_tree.subdirs[-1].pages

[DocPage(path=Path('98-reference/10-@sveltejs-kit.md'), name='@sveltejs∕kit-api-reference', digest="## Core Exports\n\n**Server class**: Handles SSR with `constructor(manifest)`, `init(options)`, and `respond(request, options)` methods.\n\n**Response helpers**:\n- `json(data, init?)` - Create JSON Response\n- `text(body, init?)` - Create text Response\n- `error(status, body)` - Throw HTTP error (prevents handleError hook)\n- `redirect(status, location)` - Throw redirect (status 300-308)\n- `fail(status, data?)` - Create ActionFailure for form handlers\n\n**Type guards**:\n- `isHttpError(e, status?)` - Check if error thrown by error()\n- `isRedirect(e)` - Check if redirect thrown by redirect()\n- `isActionFailure(e)` - Check if failure thrown by fail()\n\n**Utilities**:\n- `normalizeUrl(url)` - Strip SvelteKit suffixes and trailing slashes, returns `{url, wasNormalized, denormalize}`\n- `VERSION` - Package version string\n\n## Request/Response Types\n\n**RequestEvent**: Available in loa